# 1.3_data_modeling_postgres_fact_dimensions_star_schema
<img src="https://upload.wikimedia.org/wikipedia/commons/2/29/Postgresql_elephant.svg" width="100" height="100">

In [ ]:
# Import libraries
import psycopg2
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
dotenv_path = "../.env"
load_dotenv()

# Retrieve database connection details from the .env file
user = os.getenv("postgres_username")
password = os.getenv("postgres_password")

# Connect to database
try: 
    conn = psycopg2.connect(f"host=127.0.0.1 dbname=music user={user} password={password}")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the database")
    print(e)

# Get a cursor
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the database")
    print(e)

# Set automatic commit to be true
conn.set_session(autocommit=True)

### Imagine you're a database engineer for an online Music Store. 
The database has many tables, but let's focus on 4 tables around customer purchases.

<img src="images/1.3_data_modeling_postgres_fact_dimensions_star_schema.jpg" width="50%">

### From this representation you can start to see the makings of a "STAR". 
You will have one fact table (the center of the star) and 3  dimension tables that are coming from it. Let's create them.

In [ ]:
# CREATE fact TABLE
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS customer_transactions (customer_id int, store_id int, spent numeric)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
# INSERT INTO
try: 
    cur.execute ("INSERT INTO customer_transactions \
    VALUES (%s, %s, %s)", \
    (1, 1, 20.50))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute ("INSERT INTO customer_transactions \
    VALUES (%s, %s, %s)", \
    (2, 1, 35.21))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

In [ ]:
# CREATE dimension TABLE
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS customer (customer_id int, name varchar, rewards boolean)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS store (store_id int, state varchar)")
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS items_purchased (customer_id int, item_number int, item_name varchar)")
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
# INSERT INTO
try: 
    cur.execute ("INSERT INTO customer \
    VALUES (%s, %s, %s)", \
    (1, "Amanda", True))
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
try: 
    cur.execute ("INSERT INTO customer \
    VALUES (%s, %s, %s)", \
    (2, "Toby", False))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute ("INSERT INTO store \
    VALUES (%s, %s)", \
    (1, "CA"))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute ("INSERT INTO store \
    VALUES (%s, %s)", \
    (2, "WA"))
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
try: 
    cur.execute ("INSERT INTO items_purchased \
    VALUES (%s, %s, %s)", \
    (1, 1, "Rubber Soul"))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute ("INSERT INTO items_purchased \
    VALUES (%s, %s, %s)", \
    (2, 3, "Let It Be"))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

Now run the following queries on this data easily because of utilizing the Fact/ Dimension and Star Schema
 
- Query 1: Find all the customers that spent more than 30 dollars, who are they, which store they bought it from, location of the store, what they bought and if they are a rewards member.
    - It should be ('Toby', 1, 'CA', 'Let It Be', False)

- Query 2: How much did Customer 2 spend?
    - It should be (2, 35.21)

In [ ]:
# Query 1:
try: 
    cur.execute("SELECT name, customer_transactions.store_id, state, item_name, rewards \
    FROM (((customer_transactions \
    JOIN customer ON customer_transactions.customer_id = customer.customer_id) \
    JOIN store ON customer_transactions.store_id = store.store_id) \
    JOIN items_purchased ON customer_transactions.customer_id = items_purchased.customer_id) \
    WHERE spent > 30;")

except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

In [ ]:
# Query 2:
try: 
    cur.execute("SELECT customer_id, SUM(spent) FROM customer_transactions \
                 WHERE customer_id = 2 GROUP BY customer_id;")

except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

# Summary 
You can see here from this schema that we were: 
1. able to get "facts/metrics" from our fact table (how much each store sold), and
2. information about our customers that will allow us to do more indepth analytics to get answers to business questions by utilizing our fact and dimension tables.

Self-note: with that many JOINs in the query 1, I don't consider this a fully denormalized star schema, but the school provided this notebook.

In [ ]:
# DROP TABLE
try: 
    cur.execute("DROP table customer_transactions")    
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)

try: 
    cur.execute("DROP table customer")    
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)

try: 
    cur.execute("DROP table store")    
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)

try: 
    cur.execute("DROP table items_purchased")    
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)

In [ ]:
# cursor, connection close
try: 
    cur.close()
    conn.close()
except psycopg2.Error as e:
    print(e)